In [ ]:
from GraphModule.gcn import MyGCN
from GraphModule.nettack import Nettack

In [ ]:
#import pandas
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy import sparse

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from deeprobust.graph.defense import GCN
#from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import accuracy
from deeprobust.graph.data import Dataset

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def test_MyGCN(target_node, ori_adj, aug_adj, features, labels, train_idx, val_idx, test_idx):
    ''' test on GCN '''
    my_gcn = MyGCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device=device)

    my_gcn = my_gcn.to(device)

    my_gcn.fit(features, ori_adj, aug_adj, labels, train_idx, val_idx, patience=30)

    my_gcn.eval()
    output = my_gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('Target node probs: {}'.format(probs.detach().cpu().numpy()))
    print('Target node classification: {}'.format(probs.detach().cpu().numpy().argmax()))
    print('Target node labels: {}'.format(labels[target_node]))
    acc_test = accuracy(output[test_idx], labels[test_idx])

    print("Overall test set results:", "accuracy= {:.4f}".format(acc_test.item()))
    return acc_test.item()

In [ ]:
data = Dataset(root='Dataset', name='cora', seed=15)
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_value, idx_test = data.idx_train, data.idx_val, data.idx_test

idx_unlabeled = np.union1d(idx_value, idx_test)
idx_used = np.union1d(idx_train, idx_unlabeled)
target_node = 0
assert target_node in idx_unlabeled

In [ ]:
#defense_nodes = [172, 945, 1702, 978, 2455, 1948, 2450, 1668]
defense_nodes = [381, 894, 518, 43, 2082, 2332, 851, 216, 424]
#print(idx_train)
arg_adj = adj.todense()
#print(arg_adj[target_node].shape)
for node in defense_nodes[:5]:
    arg_adj[target_node,node] = abs( 1 - arg_adj[target_node,node])

arg_adj = sparse.csr_matrix(arg_adj)
print(adj[0])
print()
print(arg_adj[0])

In [ ]:
test_MyGCN(target_node, adj, adj, features, labels, idx_train, idx_value, idx_test)
print()
test_MyGCN(target_node, adj, arg_adj, features, labels, idx_train, idx_value, idx_test)

In [ ]:
surrogate = MyGCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item()+1, dropout=0.5, device=device)
surrogate.fit(features, adj, arg_adj, labels, idx_train, idx_value)

In [ ]:
def test_nettack_with_adj(test_adj):
    nettack = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
    nettack = nettack.to(device)
    degrees = adj.sum(0).A1
    n_perturbations = int(degrees[target_node])

    nettack.attack(features, test_adj, labels, target_node, n_perturbations)
test_nettack_with_adj(adj)
print()
test_nettack_with_adj(arg_adj)

In [ ]:
nettack = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
nettack = nettack.to(device)

In [ ]:
degrees = adj.sum(0).A1
n_perturbations = int(degrees[target_node])

nettack.attack(features, arg_adj, labels, target_node, n_perturbations)

In [ ]:
modified_adj = nettack.modified_adj
modified_features = nettack.modified_features

In [ ]:
perb_list = []
for i in idx_train[:50]:
    nettack = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
    nettack.to(device)
    nettack.attack(features, adj, labels, i, int(degrees[target_node]), verbose=False )
    perb_list.append(nettack.structure_perturbations)

In [ ]:
value_list = []
for temp in perb_list:
    for ele in temp:
        value_list.append(ele[1])

In [ ]:
from collections import Counter

b = Counter(value_list).most_common()
print(b)